In [1]:
import h5py

In [9]:
with h5py.File('/ccn2/u/thekej/contact_data/test_contact_data.hdf5', 'r') as f:
    data_contact = f['label'][:]
    filenames_contact = [s.decode() for s in f['filenames']]
    lenghts_contact = f['lengths'][:]
    print(f.keys())

<KeysViewHDF5 ['contacts', 'filenames', 'label', 'lengths']>


In [10]:
with h5py.File('/ccn2/u/thekej/models/3d_point_cloud/ocd/test_features.hdf5','r') as hf:
    print(hf.keys())
    filenames = [h.decode().split('/')[-1][:-5] for h in hf['filenames'][:]]

<KeysViewHDF5 ['contacts', 'features', 'filenames', 'label']>


In [11]:
indices, label = [], []
for i, f in enumerate(filenames):
    index = filenames_contact.index(f)
    length = lenghts_contact[index]
    for l in range(0, length, 15):
        if l + 15 < length and l + 15 <= 225:
            indices += [(i, l)]
            local_label = data_contact[index, l:l+15]
            local_label = local_label.sum(0) > 0
            label += [local_label]


In [17]:
sum(label), len(label), indices[10]

(3064, 14018, (0, 150))

In [13]:
import numpy as np

# Original array of True and False
array = np.array(label)

# Find indices of True and False using np.argwhere
indices_trues = np.argwhere(array == True).flatten()
indices_falses = np.argwhere(array == False).flatten()

# Determine the minimum count of True and False to ensure equal sampling
min_count = min(len(indices_trues), len(indices_falses))

# Sample indices for True and False
sampled_indices_trues = np.random.choice(indices_trues, min_count, replace=False)
sampled_indices_falses = np.random.choice(indices_falses, min_count, replace=False)

# Combine sampled indices
sampled_indices = np.concatenate((sampled_indices_trues, sampled_indices_falses))

# Shuffle the combined indices to mix indices of True and False
np.random.shuffle(sampled_indices)

# Output the shuffled indices
print("Sampled Indices:", sampled_indices)


Sampled Indices: [ 8310  1584  4114 ...  5043  8703 13126]


In [14]:
 np.array(label)[sampled_indices].shape

(6128,)

In [15]:
import json
# Using the sampled indices to get the corresponding True/False values from the original array
sampled_values = np.array(indices)[sampled_indices]

# Save the sampled values to a JSON file
sampled_values_list = sampled_values.tolist()  # Convert numpy array to list for JSON serialization

# Path to save the JSON file
file_path = '/ccn2/u/thekej/models/3d_point_cloud/ocd/test_snippet_indices_150_ms.json'

# Writing JSON data to a file
with open(file_path, 'w') as file:
    json.dump(sampled_values_list, file)

file_path

'/ccn2/u/thekej/models/3d_point_cloud/ocd/test_snippet_indices_150_ms.json'